In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%cp /content/features_inception_resnet_inception_v3_finetune_COLAB.csv /content/gdrive/My\ Drive/Colab\ Notebooks/data/features_inception_resnet_inception_v3_finetune_COLAB.csv

cp: cannot stat '/content/features_inception_resnet_inception_v3_COLAB.csv': No such file or directory


In [2]:
#path that contains folder you want to copy
%cp /content/gdrive/My\ Drive/Colab\ Notebooks/data/food.zip /content/
%cp /content/gdrive/My\ Drive/Colab\ Notebooks/data/train_triplets.txt /content/
%cp /content/gdrive/My\ Drive/Colab\ Notebooks/data/test_triplets.txt /content/
! unzip food.zip

Streaming output truncated to the last 5000 lines.
  inflating: food/09349.jpg          
  inflating: food/02663.jpg          
  inflating: food/03923.jpg          
  inflating: food/04639.jpg          
  inflating: food/04164.jpg          
  inflating: food/04767.jpg          
  inflating: food/09364.jpg          
  inflating: food/03961.jpg          
  inflating: food/05652.jpg          
  inflating: food/09751.jpg          
  inflating: food/02396.jpg          
  inflating: food/00182.jpg          
  inflating: food/08642.jpg          
  inflating: food/08727.jpg          
  inflating: food/09881.jpg          
  inflating: food/08958.jpg          
  inflating: food/05034.jpg          
  inflating: food/05579.jpg          
  inflating: food/01606.jpg          
  inflating: food/09424.jpg          
  inflating: food/00756.jpg          
  inflating: food/02991.jpg          
  inflating: food/02451.jpg          
  inflating: food/02701.jpg          
  inflating: food/09823.jpg          

In [0]:
import os
import pathlib
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from math import ceil, floor
from timeit import default_timer as timer
from PIL import Image
# Save image in set directory
# Read RGB image


In [4]:
np.random.seed(400)
shuffle = True
AUTOTUNE = tf.data.experimental.AUTOTUNE
print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("Availables GPU:")
print([dev.physical_device_desc for dev in tf.python.client.device_lib.list_local_devices()])
print('Gpu:' + tf.python.client.device_lib.list_local_devices()[-1].physical_device_desc)
#os.environ["TFHUB_CACHE_DIR"] = "C:/Users/Ennio/AppData/Local/Temp/model"

TF version: 2.2.0
Hub version: 0.8.0
Availables GPU:
['', 'device: XLA_CPU device', 'device: XLA_GPU device', 'device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0']
Gpu:device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [5]:
# read triplets
train_triplets_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/data/train_triplets.txt', delimiter=' ', header=None)
test_triplets_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/data/test_triplets.txt', delimiter=' ', header=None)
train_triplets_df.columns = ['A', 'B', 'C']
test_triplets_df.columns = ['A', 'B', 'C']
N_train = len(train_triplets_df.index)
N_test = len(test_triplets_df.index)

handle_base, pixels = ("inception_v3", 299)
MODULE_HANDLE = "https://tfhub.dev/google/imagenet/inception_resnet_v2/classification/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {}".format(MODULE_HANDLE, IMAGE_SIZE))

Using https://tfhub.dev/google/imagenet/inception_resnet_v2/classification/4 with input size (299, 299)


In [6]:

# build the model draft1
print("Building model with", MODULE_HANDLE)

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable=True, name='layer_A'),
    #tf.keras.layers.Dropout(rate=0.2),
    #tf.keras.layers.Dense(2,
    #                     kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])

model.build((None,)+IMAGE_SIZE+(3,))

model.summary()

Building model with https://tfhub.dev/google/imagenet/inception_resnet_v2/classification/4
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_A (KerasLayer)         (None, 1001)              55875273  
Total params: 55,875,273
Trainable params: 55,814,729
Non-trainable params: 60,544
_________________________________________________________________


In [7]:

#read images
def label2path(label):
    return '/content/food/' + str(label).zfill(5) + '.jpg'
BATCH_SIZE = 1000

features = np.zeros([10000, 1001])
for b in range(int(10000/BATCH_SIZE)):
    batch_images = np.zeros([BATCH_SIZE, pixels, pixels, 3])
    for label in range(BATCH_SIZE):
        image = Image.open(label2path(b*BATCH_SIZE+label)).resize((pixels, pixels))
        batch_images[label,:,:,:] = image
        if label%100==0:
            print(b*BATCH_SIZE+label)
    features[b*BATCH_SIZE:(b+1)*BATCH_SIZE,:] = model.predict(batch_images/255)
    


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [0]:


pd.DataFrame(data=features, columns=None, index=None).to_csv("features_inception_resnet_inception_v3_finetune_COLAB.csv", index=None, header=None,
                                                             float_format='%.8f')
